# Exemplos de uso

### Utilizando uma CNN para classificar dígitos escritos manualmente

Rede Neural Convolucional para obter uma melhor acurácia do que utilizando apenas Keras.

***Importando numpy para definer semente para o número randomico.***

In [1]:
from numpy.random import seed

***Definindo semente e importando biblioteca do tensorflow para definir semente.***

In [2]:
import tensorflow

seed(1)

tensorflow.random.set_seed(1)

***Importando bibliotecas Keras***

In [3]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten

from keras.utils import np_utils

Using TensorFlow backend.


***Carregando dados de teste e treinamento***

In [4]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

É necessário que os dados sejam ajustados a fim de permitir sua utilização. 

X_train contém 60.000 imagens de 28x28 pixels e X_test contém 10.000.

***Ajustando os dados para fragmentos de 28x28***

In [5]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

Definindo o modelo da rede, que será duas camadas convolucionais, uma camada de agrupamento do tipo max pooling, e duas camadas totalmente conectadas. 

Além disso, é necessário utilizar uma Flatten entre a camada de agrupamento e a camada totalmente conectada, pois essa espera receber uma entrada unidimensional e a saída da camada convolucional é tridimensional:

***Definindo camadas no modelo de rede***

In [6]:
model = Sequential([
    # Primeira camada convolucional
    Convolution2D(filters=32,
    kernel_size=(3, 3),
    input_shape=(28, 28, 1)),
    Activation('relu'),

    # Segunda camada convolucional
    Convolution2D(filters=32,
    kernel_size=(3, 3)),
    Activation('relu'),

    # Camada de agrupamento (Max pooling)
    MaxPooling2D(pool_size=(2, 2)),

    # Transformando o tensor em unidimensional
    Flatten(),

    #Camada oculta totalmente conectada
    Dense(64),
    Activation('relu'),

    # Camada de saída
    Dense(10),
    Activation('softmax')
])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

Em seguida, será definido o otimizador. Ao invés do Gradiente Descendente Estocástico (SGD), utilizaremos o ADADELTA, que automaticamente definirá uma taxa de aprendizado maior ou menor de forma inversamente proporcional ao gradiente:

***Compilando o modelo com otimização em ADADELTA***

In [7]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adadelta')

***Treinando o modelo***

In [8]:
model.fit(X_train, Y_train, batch_size=100, epochs=5, validation_split=0.1, verbose=1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 41s 762us/step - loss: 0.5327 - accuracy: 0.9329 - val_loss: 0.0743 - val_accuracy: 0.9770
Epoch 2/5
54000/54000 [==============================] - 39s 728us/step - loss: 0.0560 - accuracy: 0.9830 - val_loss: 0.0570 - val_accuracy: 0.9848
Epoch 3/5
54000/54000 [==============================] - 37s 684us/step - loss: 0.0288 - accuracy: 0.9916 - val_loss: 0.0433 - val_accuracy: 0.9880
Epoch 4/5
54000/54000 [==============================] - 35s 642us/step - loss: 0.0165 - accuracy: 0.9948 - val_loss: 0.0426 - val_accuracy: 0.9895
Epoch 5/5
54000/54000 [==============================] - 36s 670us/step - loss: 0.0085 - accuracy: 0.9971 - val_loss: 0.0562 - val_accuracy: 0.9892


***Testando a rede***

In [9]:
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test accuracy:', score[1])

10000/10000 [==============================] - 20s 2ms/step
Test accuracy: 0.988099992275238


Foi obtida uma acurácia de 98,5% utilizando este modelo, contra 96% de acurácia utilizando uma Rede Neural simples.